In [1]:
# Import basic packages
import numpy as np
import scipy as sp
import pandas as pd
from astropy.io import fits


# ==== Scikit-learn =======================
# Preprocessing
from sklearn.preprocessing import StandardScaler #Standar scaler for standardization
from sklearn.model_selection import train_test_split # For random split
from sklearn.utils import resample #for jack knife resampling

# Classifiers
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint

# ==========================================
# Matplotlib, urlib etc 
import urllib
import urllib.request
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from PIL import Image
%matplotlib inline

In [2]:
from sklearn.metrics import confusion_matrix

def rates(y_true, y_pred):
    CM = confusion_matrix(y_true, y_pred)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    # Initialize
    X_mat = np.zeros(4)
    
    # Populate
    X_mat[0] = TN
    X_mat[1] = FN
    X_mat[2] = TP
    X_mat[3] = FP
    
    return X_mat  

### Import feature matrix and labels

In [3]:
randoms = np.load("/data/des81.a/data/kherron/LSBG/trainingfiles/Randoms/X_randoms_feat.npy")
randoms_l = np.load("/data/des81.a/data/kherron/LSBG/trainingfiles/Randoms/y_randoms_lab.npy")


burcin = np.load("/data/des80.b/data/burcinmp/y6_lsbg/y6/test_classifier/random_forest/v3/X_mat_v4_a.npy")
burcin_l = np.load("/data/des80.b/data/burcinmp/y6_lsbg/y6/test_classifier/random_forest/v3/y_lab_v4_a.npy")


sel = (burcin_l == 1)
burcin = burcin[sel]
burcin_l = burcin_l[sel]

In [22]:
X_feat_real = np.concatenate((randoms,burcin))
y_lab_real = np.concatenate((randoms_l,burcin_l))

X_feat_y3 = np.load("/data/des81.a/data/kherron/LSBG/trainingfiles/X_mat_y3.npy") 
y_lab_y3 = np.load("/data/des81.a/data/kherron/LSBG/trainingfiles/y_lab_y3.npy")

X_feat_sim = np.load("/data/des81.a/data/kherron/LSBG/trainingfiles/X_mat_sim_clean.npy") 
y_lab_sim = np.load("/data/des81.a/data/kherron/LSBG/trainingfiles/y_lab_sim_clean.npy")

print("Number of visually inspected LSBGs: ",len(y_lab_real[y_lab_real==1]))
print("Y3 LSBGs: ",len(y_lab_y3))
print("Simulated LSBGs: ",len(y_lab_sim[y_lab_sim==1]))

Number of visually inspected LSBGs:  30475
Y3 LSBGs:  20331
Simulated LSBGs:  8154


In [31]:
X_feat_real, y_lab_real = resample((y_lab_real, X_feat_real), replace=False, random_state=42)

### Create the final model and evaluate the performance

In [32]:
index_real=(y_lab_real==1)
index_art=(y_lab_real==0)

X_art = X_feat_real[index_art]
X_lsbg = X_feat_real[index_real]
X_sim = X_feat_sim

y_art = y_lab_real[index_art]
y_lsbg = y_lab_real[index_real]
y_sim = y_lab_sim

X_train_art, X_test_art, y_train_art, y_test_art = train_test_split(X_art, y_art,
                                                                        train_size = 0.70, random_state = 42)
X_train_lsbg, X_test_lsbg, y_train_lsbg, y_test_lsbg = train_test_split(X_lsbg, y_lsbg,
                                                                        train_size = 0.70, random_state = 42)
X_train_sim, X_test_sim, y_train_sim, y_test_sim = train_test_split(X_sim, y_sim,
                                                                        train_size = 0.70, random_state = 42)
X_train_y3, X_test_y3, y_train_y3, y_test_y3 = train_test_split(X_feat_y3, y_lab_y3,
                                                                        train_size = 0.70, random_state = 42)

print("ARTIFACTS:",len(X_train_art), len(X_test_art))
print("Y3:",len(X_train_y3), len(X_test_y3))
print("LSBGs:",len(X_train_lsbg), len(X_test_lsbg))
print("SIMs:",len(X_train_sim), len(X_test_sim))


ARTIFACTS: 70000 30000
Y3: 14231 6100
LSBGs: 21332 9143
SIMs: 5707 2447


In [33]:
#X_train = np.concatenate((X_train_art,X_train_lsbg,X_train_y3))#,X_train_sim))
#y_train = np.concatenate((y_train_art,y_train_lsbg,y_train_y3))#,y_train_sim))

X_train = np.concatenate((X_train_art,X_train_lsbg))#,X_train_sim))
y_train = np.concatenate((y_train_art,y_train_lsbg))#,y_train_sim))

# Standardize the two sets
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

In [34]:
#Using Default
model_rf_fin = RandomForestClassifier(n_estimators=100)
#                                       criterion='entropy',
#                                       max_depth=10,
#                                       max_features=None)
#                                       min_samples_leaf=2,
#                                       min_samples_split=12)
#                                       n_estimators=240)
model_rf_fin.fit(X_train,y_train)


RandomForestClassifier()

In [35]:
print('AUC Score ={:.4f}'.format(metrics.f1_score(y_train,model_rf_fin.predict(X_train))))

AUC Score =0.9884


In [36]:
X_mat_save_a = np.concatenate((X_art,X_lsbg,X_feat_y3,X_sim))
y_lab_save_a = np.concatenate((y_art,y_lsbg,y_lab_y3,y_sim))
np.save('X_mat_v4_a',X_mat_save_a)
np.save('y_lab_v4_a',y_lab_save_a)

X_mat_save_b = np.concatenate((X_art,X_lsbg,X_sim))
y_lab_save_b = np.concatenate((y_art,y_lsbg,y_sim))
#np.save('X_mat_v4_b',X_mat_save_b)
#np.save('y_lab_v4_b',y_lab_save_b)

X_mat_save_c = np.concatenate((X_art,X_lsbg,X_feat_y3))
y_lab_save_c = np.concatenate((y_art,y_lsbg,y_lab_y3))
#np.save('X_mat_v4_c',X_mat_save_c)
#np.save('y_lab_v4_c',y_lab_save_c)

X_mat_save_d = np.concatenate((X_art,X_lsbg))
y_lab_save_d = np.concatenate((y_art,y_lsbg))
#np.save('X_mat_v4_d',X_mat_save_d)
#np.save('y_lab_v4_d',y_lab_save_d)


print(len(y_lab_save_a),len(y_lab_save_b),len(y_lab_save_c),len(y_lab_save_d))

158960 138629 150806 130475


# Make Test Sample

In [37]:
# With No Simulations
X_test_10 = np.concatenate((X_test_lsbg,X_test_y3,X_test_art)) 
y_test_10 = np.concatenate((y_test_lsbg,y_test_y3,y_test_art))

# With Only Simulations
X_test_20 = np.concatenate((X_test_sim,X_test_art))
y_test_20 = np.concatenate((y_test_sim,y_test_art))

print(len(X_test_10),len(X_test_20),len(X_test_art),len(X_test_lsbg),len(X_feat_y3),len(X_test_sim))

45243 32447 30000 9143 20331 2447


### Start binning

In [38]:
x_bins = [24,25,26,27,28]
X1 = np.concatenate((X_test_lsbg,X_test_y3))
y1 = np.concatenate((y_test_lsbg,y_test_y3))
bin1 = (X1[:,16]> x_bins[0]) & (X1[:,16]< x_bins[1])
bin2 = (X1[:,16]> x_bins[1]) & (X1[:,16]< x_bins[2])
bin3 = (X1[:,16]> x_bins[2]) & (X1[:,16]< x_bins[3])
bin4 = (X1[:,16]> x_bins[3]) & (X1[:,16]< x_bins[4])
bin5 = (X1[:,16]> x_bins[4])

In [39]:
X2 = X_test_sim 
y2 = y_test_sim 
bin10 = (X2[:,16]> x_bins[0]) & (X2[:,16]< x_bins[1])
bin20 = (X2[:,16]> x_bins[1]) & (X2[:,16]< x_bins[2])
bin30 = (X2[:,16]> x_bins[2]) & (X2[:,16]< x_bins[3])
bin40 = (X2[:,16]> x_bins[3]) & (X2[:,16]< x_bins[4])
bin50 = (X2[:,16]> x_bins[4])

In [40]:
print(x_bins)
print(len(X1[bin1]),len(X1[bin2]),len(X1[bin3]),len(X1[bin4]),len(X1[bin5]))
print(len(X2[bin10]),len(X2[bin20]),len(X2[bin30]),len(X2[bin40]),len(X2[bin50]))

[24, 25, 26, 27, 28]
10388 4139 701 14 1
1012 1150 285 0 0


In [41]:
# 24 <= mu_g<=25
#Select 209 from real
X1_bin1 = X1[bin1]
y1_bin1 = y1[bin1]
X2_bin1 = X2[bin10]
y2_bin1 = y2[bin10]

all1 = np.arange(len(X1_bin1)) 
ind1 = np.random.choice(all1, size=len(X2_bin1), replace=False)

X1_test_bin1 = X1_bin1[ind1]
y1_test_bin1 =  y1_bin1[ind1]
X2_test_bin1 = X2_bin1
y2_test_bin1 =  y2_bin1

print(len(X1_test_bin1))

1012


In [42]:
# 25. <= mu_g<= 26
# Select 1186 from real
X1_bin2 = X1[bin2]
y1_bin2 = y1[bin2]
X2_bin2 = X2[bin20]
y2_bin2 = y2[bin20]

all2 = np.arange(len(X1_bin2)) 
ind2 = np.random.choice(all2, size=len(X2_bin2), replace=False)

X1_test_bin2 = X1_bin2[ind2]
y1_test_bin2 =  y1_bin2[ind2]
X2_test_bin2 = X2_bin2
y2_test_bin2 =  y2_bin2

print(len(X2_test_bin2))

1150


In [43]:
# 26. <= mu_g<= 27
# Select 1052 from real
X1_bin3 = X1[bin3]
y1_bin3 = y1[bin3]
X2_bin3 = X2[bin30]
y2_bin3 = y2[bin30]

all3 = np.arange(len(X1_bin3)) 
ind3 = np.random.choice(all3, size=len(X2_bin3), replace=False)

X1_test_bin3 = X1_bin3[ind3]
y1_test_bin3 =  y1_bin3[ind3]
X2_test_bin3 = X2_bin3
y2_test_bin3 =  y2_bin3

print(len(X1_test_bin3))

285


### Binned test set

In [44]:
#artifacts accordingly
xx1 = len(X1_test_bin1)
xx2 = len(X1_test_bin2)
xx3 = len(X1_test_bin3)
num_a1 = 20*xx1
num_a2 = 20*xx2
num_a3 = 20*xx3

all4 = np.arange(len(X_art)) 
ind4_1 = np.random.choice(all4, size=num_a1, replace=False)
ind4_2 = np.random.choice(all4, size=num_a2, replace=False)
ind4_3 = np.random.choice(all4, size=num_a3, replace=False)

X_test_art1 = X_art[ind4_1]
y_test_art1 =  y_art[ind4_1]
X_test_art2 = X_art[ind4_2]
y_test_art2 =  y_art[ind4_2]
X_test_art3 = X_art[ind4_3]
y_test_art3 =  y_art[ind4_3]

print(len(X_test_art1),len(X_test_art2),len(X_test_art3))

20240 23000 5700


In [45]:
print(xx1)

1012


In [46]:
# Only real data
X_test_bin_1 = np.concatenate((X_test_art,X1_test_bin1,X1_test_bin2,X1_test_bin3)) 
y_test_bin_1 = np.concatenate((y_test_art,y1_test_bin1,y1_test_bin2,y1_test_bin3))
# Only simulation data
X_test_bin_2 = np.concatenate((X_test_art,X2_test_bin1,X2_test_bin2,X2_test_bin3)) 
y_test_bin_2 = np.concatenate((y_test_art,y2_test_bin1,y2_test_bin2,y2_test_bin3))

X_test_bin_1 = scaler.transform(X_test_bin_1)
X_test_bin_2 = scaler.transform(X_test_bin_2)

print(len(X_test_art),len(X1_test_bin1)+len(X1_test_bin2)+len(X1_test_bin3))

30000 2447


### BIN1: 24. <= mu_g<= 25

In [47]:
# Binned test set
# Only real data
X_test_bin1_1 = np.concatenate((X_test_art1,X1_test_bin1)) 
y_test_bin1_1 = np.concatenate((y_test_art1,y1_test_bin1))
# Only simulation data
X_test_bin1_2 = np.concatenate((X_test_art1,X2_test_bin1)) 
y_test_bin1_2 = np.concatenate((y_test_art1,y2_test_bin1))

X_test_bin1_1 = scaler.transform(X_test_bin1_1)
X_test_bin1_2 = scaler.transform(X_test_bin1_2)

# Predict on the test set
y_pred_bin1_1 = model_rf_fin.predict(X_test_bin1_1)
y_pred_bin1_2 = model_rf_fin.predict(X_test_bin1_2)

print(len(X_test_art1),len(X1_test_bin1),len(X2_test_bin1))

20240 1012 1012


In [48]:
print("Precision (purity):",precision_score(y_test_bin1_1,y_pred_bin1_1))
print("Recall (completeness):",recall_score(y_test_bin1_1,y_pred_bin1_1))
print("Balanced accuracy:", balanced_accuracy_score(y_test_bin1_1,y_pred_bin1_1))

Precision (purity): 0.782051282051282
Recall (completeness): 0.9644268774703557
Balanced accuracy: 0.975494071146245


In [49]:
print("Precision (purity):",precision_score(y_test_bin1_2,y_pred_bin1_2))
print("Recall (completeness):",recall_score(y_test_bin1_2,y_pred_bin1_2))
print("Balanced accuracy:", balanced_accuracy_score(y_test_bin1_2,y_pred_bin1_2))

Precision (purity): 0.7871674491392802
Recall (completeness): 0.9940711462450593
Balanced accuracy: 0.9903162055335968


In [50]:
rate_bin1_1 = rates(y_test_bin1_1,y_pred_bin1_1)
rate_bin1_2 = rates(y_test_bin1_2,y_pred_bin1_2)

print("TN   FN   TP   FP")
print(rate_bin1_1)
print(rate_bin1_2)

TN   FN   TP   FP
[19968.    36.   976.   272.]
[1.9968e+04 6.0000e+00 1.0060e+03 2.7200e+02]


In [51]:
TN = rate_bin1_1[0]
FN = rate_bin1_1[1]
TP = rate_bin1_1[2]
FP = rate_bin1_1[3]

# Sensitivity, hit rate, recall, or true positive rate
TPR_bin1_1 = TP/(TP+FN)
print("true positive rate:",TPR_bin1_1)
# Specificity or true negative rate
TNR_bin1_1 = TN/(TN+FP) 
print("true negative rate:",TNR_bin1_1)
# Precision or positive predictive value
PPV_bin1_1 = TP/(TP+FP)
print("positive predictive value:",PPV_bin1_1)
# Negative predictive value
NPV_bin1_1 = TN/(TN+FN)
print("negative predictive value:",NPV_bin1_1)
# Fall out or false positive rate
FPR_bin1_1 = FP/(FP+TN)
print("false positive rate:",FPR_bin1_1)
# False negative rate
FNR_bin1_1 = FN/(TP+FN)
print("false negative rate:",FNR_bin1_1)
# False discovery rate
FDR_bin1_1 = FP/(TP+FP)
print("false discovery rate:",FDR_bin1_1)

# Overall accuracy
ACC_bin1_1 = (TP+TN)/(TP+FP+FN+TN)
print("Overall accuracy:",ACC_bin1_1)

true positive rate: 0.9644268774703557
true negative rate: 0.9865612648221344
positive predictive value: 0.782051282051282
negative predictive value: 0.9982003599280144
false positive rate: 0.013438735177865613
false negative rate: 0.03557312252964427
false discovery rate: 0.21794871794871795
Overall accuracy: 0.9855072463768116


### BIN2: 25. <= mu_g<= 26

In [52]:
# Binned test set
# Only real data
X_test_bin2_1 = np.concatenate((X_test_art2,X1_test_bin2)) 
y_test_bin2_1 = np.concatenate((y_test_art2,y1_test_bin2))
# Only simulation data
X_test_bin2_2 = np.concatenate((X_test_art2,X2_test_bin2)) 
y_test_bin2_2 = np.concatenate((y_test_art2,y2_test_bin2))

X_test_bin2_1 = scaler.transform(X_test_bin2_1)
X_test_bin2_2 = scaler.transform(X_test_bin2_2)

# Predict on the test set
y_pred_bin2_1 = model_rf_fin.predict(X_test_bin2_1)
y_pred_bin2_2 = model_rf_fin.predict(X_test_bin2_2)

print(len(X_test_art2),len(X1_test_bin2),len(X2_test_bin2))

23000 1150 1150


In [53]:
print("Precision (purity):",precision_score(y_test_bin2_1,y_pred_bin2_1))
print("Recall (completeness):",recall_score(y_test_bin2_1,y_pred_bin2_1))
print("Balanced accuracy:", balanced_accuracy_score(y_test_bin2_1,y_pred_bin2_1))

Precision (purity): 0.7916964924838941
Recall (completeness): 0.9617391304347827
Balanced accuracy: 0.9745434782608695


In [54]:
print("Precision (purity):",precision_score(y_test_bin2_2,y_pred_bin2_2))
print("Recall (completeness):",recall_score(y_test_bin2_2,y_pred_bin2_2))
print("Balanced accuracy:", balanced_accuracy_score(y_test_bin2_2,y_pred_bin2_2))

Precision (purity): 0.7960756832515767
Recall (completeness): 0.9878260869565217
Balanced accuracy: 0.9875869565217391


In [55]:
rate_bin2_1 = rates(y_test_bin2_1,y_pred_bin2_1)
rate_bin2_2 = rates(y_test_bin2_2,y_pred_bin2_2)

print("TN   FN   TP   FP")
print(rate_bin2_1)
print(rate_bin2_2)

TN   FN   TP   FP
[22709.    44.  1106.   291.]
[2.2709e+04 1.4000e+01 1.1360e+03 2.9100e+02]


In [56]:
TN = rate_bin2_1[0]
FN = rate_bin2_1[1]
TP = rate_bin2_1[2]
FP = rate_bin2_1[3]

# Sensitivity, hit rate, recall, or true positive rate
TPR_bin2_1 = TP/(TP+FN)
print("true positive rate:",TPR_bin2_1)
# Specificity or true negative rate
TNR_bin2_1 = TN/(TN+FP) 
print("true negative rate:",TNR_bin2_1)
# Precision or positive predictive value
PPV_bin2_1 = TP/(TP+FP)
print("positive predictive value:",PPV_bin2_1)
# Negative predictive value
NPV_bin2_1 = TN/(TN+FN)
print("negative predictive value:",NPV_bin2_1)
# Fall out or false positive rate
FPR_bin2_1 = FP/(FP+TN)
print("false positive rate:",FPR_bin2_1)
# False negative rate
FNR_bin2_1 = FN/(TP+FN)
print("false negative rate:",FNR_bin2_1)
# False discovery rate
FDR_bin2_1 = FP/(TP+FP)
print("false discovery rate:",FDR_bin2_1)

# Overall accuracy
ACC_bin2_1 = (TP+TN)/(TP+FP+FN+TN)
print("Overall accuracy:",ACC_bin2_1)

true positive rate: 0.9617391304347827
true negative rate: 0.9873478260869565
positive predictive value: 0.7916964924838941
negative predictive value: 0.9980661890739683
false positive rate: 0.012652173913043479
false negative rate: 0.03826086956521739
false discovery rate: 0.20830350751610593
Overall accuracy: 0.9861283643892339


### BIN3: 26. <= mu_g<= 27

In [57]:
# Binned test set
# Only real data
X_test_bin3_1 = np.concatenate((X_test_art3,X1_test_bin3)) 
y_test_bin3_1 = np.concatenate((y_test_art3,y1_test_bin3))
# Only simulation data
X_test_bin3_2 = np.concatenate((X_test_art3,X2_test_bin3)) 
y_test_bin3_2 = np.concatenate((y_test_art3,y2_test_bin3))

X_test_bin3_1 = scaler.transform(X_test_bin3_1)
X_test_bin3_2 = scaler.transform(X_test_bin3_2)

# Predict on the test set
y_pred_bin3_1 = model_rf_fin.predict(X_test_bin3_1)
y_pred_bin3_2 = model_rf_fin.predict(X_test_bin3_2)

print(len(X_test_art3),len(X1_test_bin3),len(X2_test_bin3))

5700 285 285


In [58]:
print("Precision (purity):",precision_score(y_test_bin3_1,y_pred_bin3_1))
print("Recall (completeness):",recall_score(y_test_bin3_1,y_pred_bin3_1))
print("Balanced accuracy:", balanced_accuracy_score(y_test_bin3_1,y_pred_bin3_1))

Precision (purity): 0.7692307692307693
Recall (completeness): 0.8070175438596491
Balanced accuracy: 0.8974561403508772


In [59]:
print("Precision (purity):",precision_score(y_test_bin3_2,y_pred_bin3_2))
print("Recall (completeness):",recall_score(y_test_bin3_2,y_pred_bin3_2))
print("Balanced accuracy:", balanced_accuracy_score(y_test_bin3_2,y_pred_bin3_2))

Precision (purity): 0.8
Recall (completeness): 0.968421052631579
Balanced accuracy: 0.9781578947368421


In [60]:
rate_bin3_1 = rates(y_test_bin3_1,y_pred_bin3_1)
rate_bin3_2 = rates(y_test_bin3_2,y_pred_bin3_2)

print("TN   FN   TP   FP")
print(rate_bin3_1)
print(rate_bin3_2)

TN   FN   TP   FP
[5631.   55.  230.   69.]
[5631.    9.  276.   69.]


In [61]:
TN = rate_bin3_1[0]
FN = rate_bin3_1[1]
TP = rate_bin3_1[2]
FP = rate_bin3_1[3]

# Sensitivity, hit rate, recall, or true positive rate
TPR_bin3_1 = TP/(TP+FN)
print("true positive rate:",TPR_bin3_1)
# Specificity or true negative rate
TNR_bin3_1 = TN/(TN+FP) 
print("true negative rate:",TNR_bin3_1)
# Precision or positive predictive value
PPV_bin3_1 = TP/(TP+FP)
print("positive predictive value:",PPV_bin3_1)
# Negative predictive value
NPV_bin3_1 = TN/(TN+FN)
print("negative predictive value:",NPV_bin3_1)
# Fall out or false positive rate
FPR_bin3_1 = FP/(FP+TN)
print("false positive rate:",FPR_bin3_1)
# False negative rate
FNR_bin3_1 = FN/(TP+FN)
print("false negative rate:",FNR_bin3_1)
# False discovery rate
FDR_bin3_1 = FP/(TP+FP)
print("false discovery rate:",FDR_bin3_1)

# Overall accuracy
ACC_bin3_1 = (TP+TN)/(TP+FP+FN+TN)
print("Overall accuracy:",ACC_bin3_1)

true positive rate: 0.8070175438596491
true negative rate: 0.9878947368421053
positive predictive value: 0.7692307692307693
negative predictive value: 0.9903271192402392
false positive rate: 0.012105263157894737
false negative rate: 0.19298245614035087
false discovery rate: 0.23076923076923078
Overall accuracy: 0.979281537176274
